In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib, pymysql, calendar, time, json
from urllib.request import urlopen
from datetime import datetime
from threading import Timer

class DBUpdater:  
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성"""
        self.conn = pymysql.connect(host='localhost', user='root',
            password='1234', db='INVESTAR', charset='utf8')
        
        with self.conn.cursor() as curs:
            sql = """
            CREATE TABLE IF NOT EXISTS company_info (
                code VARCHAR(20),
                company VARCHAR(40),
                last_update DATE,
                PRIMARY KEY (code))
            """
            curs.execute(sql)
            sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))
            """
            curs.execute(sql)
        self.conn.commit()
        self.codes = dict()
               
    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
        self.conn.close() 
     
    def read_krx_code(self):
        """KRX로부터 상장기업 목록 파일을 읽어와서 데이터프레임으로 반환"""
        url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
            'download&searchType=13'
        krx = pd.read_html(url, header=0)[0]
        krx = krx[['종목코드', '회사명']]
        krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
        krx.code = krx.code.map('{:06d}'.format)
        return krx
    
    def update_comp_info(self):
        """종목코드를 company_info 테이블에 업데이트 한 후 딕셔너리에 저장"""
        sql = "SELECT * FROM company_info"
        df = pd.read_sql(sql, self.conn)
        for idx in range(len(df)):
            self.codes[df['code'].values[idx]] = df['company'].values[idx]
                    
        with self.conn.cursor() as curs:
            sql = "SELECT max(last_update) FROM company_info"
            curs.execute(sql)
            rs = curs.fetchone()
            today = datetime.today().strftime('%Y-%m-%d')
            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:
                krx = self.read_krx_code()
                for idx in range(len(krx)):
                    code = krx.code.values[idx]
                    company = krx.company.values[idx]                
                    sql = f"REPLACE INTO company_info (code, company, last"\
                        f"_update) VALUES ('{code}', '{company}', '{today}')"
                    curs.execute(sql)
                    self.codes[code] = company
                    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                    print(f"[{tmnow}] #{idx+1:04d} REPLACE INTO company_info "\
                        f"VALUES ({code}, {company}, {today})")
                self.conn.commit()
                print('')              

    def read_naver(self, code, company, pages_to_fetch):
        """네이버에서 주식 시세를 읽어서 데이터프레임으로 반환"""
        try:
            url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
            with urlopen(url) as doc:
                if doc is None:
                    return None
                html = BeautifulSoup(doc, "lxml")
                pgrr = html.find("td", class_="pgRR")
                if pgrr is None:
                    return None
                s = str(pgrr.a["href"]).split('=')
                lastpage = s[-1] 
            df = pd.DataFrame()
            pages = min(int(lastpage), pages_to_fetch)
            for page in range(1, pages + 1):
                pg_url = '{}&page={}'.format(url, page)
                df = df.append(pd.read_html(pg_url, header=0)[0])
                tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.
                    format(tmnow, company, code, page, pages), end="\r")
            df = df.rename(columns={'날짜':'date','종가':'close','전일비':'diff'
                ,'시가':'open','고가':'high','저가':'low','거래량':'volume'})
            df['date'] = df['date'].replace('.', '-')
            df = df.dropna()
            df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close',
                'diff', 'open', 'high', 'low', 'volume']].astype(int)
            df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']]
        except Exception as e:
            print('Exception occured :', str(e))
            return None
        return df

    def replace_into_db(self, df, num, code, company):
        """네이버에서 읽어온 주식 시세를 DB에 REPLACE"""
        with self.conn.cursor() as curs:
            for r in df.itertuples():
                sql = f"REPLACE INTO daily_price VALUES ('{code}', "\
                    f"'{r.date}', {r.open}, {r.high}, {r.low}, {r.close}, "\
                    f"{r.diff}, {r.volume})"
                curs.execute(sql)
            self.conn.commit()
            print('[{}] #{:04d} {} ({}) : {} rows > REPLACE INTO daily_'\
                'price [OK]'.format(datetime.now().strftime('%Y-%m-%d'\
                ' %H:%M'), num+1, company, code, len(df)))

    def update_daily_price(self, pages_to_fetch):
        """KRX 상장법인의 주식 시세를 네이버로부터 읽어서 DB에 업데이트"""  
        for idx, code in enumerate(self.codes):
            df = self.read_naver(code, self.codes[code], pages_to_fetch)
            if df is None:
                continue
            self.replace_into_db(df, idx, code, self.codes[code])            

    def execute_daily(self):
        """실행 즉시 및 매일 오후 다섯시에 daily_price 테이블 업데이트"""
        self.update_comp_info()
        
        try:
            with open('config.json', 'r') as in_file:
                config = json.load(in_file)
                pages_to_fetch = config['pages_to_fetch']
        except FileNotFoundError:
            with open('config.json', 'w') as out_file:
                # 총 610페이지
                pages_to_fetch = 50 
                config = {'pages_to_fetch': 1}
                json.dump(config, out_file)
        self.update_daily_price(pages_to_fetch)

        tmnow = datetime.now()
        lastday = calendar.monthrange(tmnow.year, tmnow.month)[1]
        if tmnow.month == 12 and tmnow.day == lastday:
            tmnext = tmnow.replace(year=tmnow.year+1, month=1, day=1,
                hour=17, minute=0, second=0)
        elif tmnow.day == lastday:
            tmnext = tmnow.replace(month=tmnow.month+1, day=1, hour=17,
                minute=0, second=0)
        else:
            tmnext = tmnow.replace(day=tmnow.day+1, hour=17, minute=0,
                second=0)   
        tmdiff = tmnext - tmnow
        secs = tmdiff.seconds
        t = Timer(secs, self.execute_daily)
        print("Waiting for next update ({}) ... ".format(tmnext.strftime
            ('%Y-%m-%d %H:%M')))
        t.start()

if __name__ == '__main__':
    dbu = DBUpdater()
    dbu.execute_daily()

[2020-10-29 12:00] #0001 REPLACE INTO company_info VALUES (155660, DSR, 2020-10-29)
[2020-10-29 12:00] #0002 REPLACE INTO company_info VALUES (001250, GS글로벌, 2020-10-29)
[2020-10-29 12:00] #0003 REPLACE INTO company_info VALUES (294870, HDC현대산업개발, 2020-10-29)
[2020-10-29 12:00] #0004 REPLACE INTO company_info VALUES (092220, KEC, 2020-10-29)
[2020-10-29 12:00] #0005 REPLACE INTO company_info VALUES (001390, KG케미칼, 2020-10-29)
[2020-10-29 12:00] #0006 REPLACE INTO company_info VALUES (011070, LG이노텍, 2020-10-29)
[2020-10-29 12:00] #0007 REPLACE INTO company_info VALUES (066570, LG전자, 2020-10-29)
[2020-10-29 12:00] #0008 REPLACE INTO company_info VALUES (037560, LG헬로비전, 2020-10-29)
[2020-10-29 12:00] #0009 REPLACE INTO company_info VALUES (010060, OCI, 2020-10-29)
[2020-10-29 12:00] #0010 REPLACE INTO company_info VALUES (100840, S&TC, 2020-10-29)
[2020-10-29 12:00] #0011 REPLACE INTO company_info VALUES (064960, S&T모티브, 2020-10-29)
[2020-10-29 12:00] #0012 REPLACE INTO company_info VALUE

[2020-10-29 12:00] #0421 REPLACE INTO company_info VALUES (014100, 메디앙스, 2020-10-29)
[2020-10-29 12:00] #0422 REPLACE INTO company_info VALUES (059210, 메타바이오메드, 2020-10-29)
[2020-10-29 12:00] #0423 REPLACE INTO company_info VALUES (058110, 멕아이씨에스, 2020-10-29)
[2020-10-29 12:00] #0424 REPLACE INTO company_info VALUES (080160, 모두투어, 2020-10-29)
[2020-10-29 12:00] #0425 REPLACE INTO company_info VALUES (001810, 무림SP, 2020-10-29)
[2020-10-29 12:00] #0426 REPLACE INTO company_info VALUES (328380, 미래에셋대우스팩3호, 2020-10-29)
[2020-10-29 12:00] #0427 REPLACE INTO company_info VALUES (100790, 미래에셋벤처투자, 2020-10-29)
[2020-10-29 12:00] #0428 REPLACE INTO company_info VALUES (059090, 미코, 2020-10-29)
[2020-10-29 12:00] #0429 REPLACE INTO company_info VALUES (301300, 바이브컴퍼니, 2020-10-29)
[2020-10-29 12:00] #0430 REPLACE INTO company_info VALUES (323990, 박셀바이오, 2020-10-29)
[2020-10-29 12:00] #0431 REPLACE INTO company_info VALUES (008470, 부스타, 2020-10-29)
[2020-10-29 12:00] #0432 REPLACE INTO company_info

[2020-10-29 12:00] #0956 REPLACE INTO company_info VALUES (002420, 세기상사, 2020-10-29)
[2020-10-29 12:00] #0957 REPLACE INTO company_info VALUES (019440, 세아특수강, 2020-10-29)
[2020-10-29 12:00] #0958 REPLACE INTO company_info VALUES (013000, 세우글로벌, 2020-10-29)
[2020-10-29 12:00] #0959 REPLACE INTO company_info VALUES (021820, 세원정공, 2020-10-29)
[2020-10-29 12:00] #0960 REPLACE INTO company_info VALUES (075580, 세진중공업, 2020-10-29)
[2020-10-29 12:00] #0961 REPLACE INTO company_info VALUES (145210, 세화아이엠씨, 2020-10-29)
[2020-10-29 12:00] #0962 REPLACE INTO company_info VALUES (068270, 셀트리온, 2020-10-29)
[2020-10-29 12:00] #0963 REPLACE INTO company_info VALUES (004970, 신라교역, 2020-10-29)
[2020-10-29 12:00] #0964 REPLACE INTO company_info VALUES (005390, 신성통상, 2020-10-29)
[2020-10-29 12:00] #0965 REPLACE INTO company_info VALUES (004170, 신세계, 2020-10-29)
[2020-10-29 12:00] #0966 REPLACE INTO company_info VALUES (090430, 아모레퍼시픽, 2020-10-29)
[2020-10-29 12:00] #0967 REPLACE INTO company_info VALUES (

[2020-10-29 12:00] #1439 REPLACE INTO company_info VALUES (073010, 케이에스피, 2020-10-29)
[2020-10-29 12:00] #1440 REPLACE INTO company_info VALUES (224060, 코디엠, 2020-10-29)
[2020-10-29 12:00] #1441 REPLACE INTO company_info VALUES (290510, 코리아센터, 2020-10-29)
[2020-10-29 12:00] #1442 REPLACE INTO company_info VALUES (115180, 큐리언트, 2020-10-29)
[2020-10-29 12:00] #1443 REPLACE INTO company_info VALUES (045520, 크린앤사이언스, 2020-10-29)
[2020-10-29 12:00] #1444 REPLACE INTO company_info VALUES (214150, 클래시스, 2020-10-29)
[2020-10-29 12:00] #1445 REPLACE INTO company_info VALUES (044490, 태웅, 2020-10-29)
[2020-10-29 12:00] #1446 REPLACE INTO company_info VALUES (182690, 테라셈, 2020-10-29)
[2020-10-29 12:00] #1447 REPLACE INTO company_info VALUES (215480, 토박스코리아, 2020-10-29)
[2020-10-29 12:00] #1448 REPLACE INTO company_info VALUES (057680, 티사이언티픽, 2020-10-29)
[2020-10-29 12:00] #1449 REPLACE INTO company_info VALUES (104480, 티케이케미칼, 2020-10-29)
[2020-10-29 12:00] #1450 REPLACE INTO company_info VALUES 

[2020-10-29 12:00] #1950 REPLACE INTO company_info VALUES (036710, 심텍홀딩스, 2020-10-29)
[2020-10-29 12:00] #1951 REPLACE INTO company_info VALUES (049960, 쎌바이오텍, 2020-10-29)
[2020-10-29 12:00] #1952 REPLACE INTO company_info VALUES (260930, 씨티케이코스메틱스, 2020-10-29)
[2020-10-29 12:00] #1953 REPLACE INTO company_info VALUES (125210, 아모그린텍, 2020-10-29)
[2020-10-29 12:00] #1954 REPLACE INTO company_info VALUES (074430, 아미노로직스, 2020-10-29)
[2020-10-29 12:00] #1955 REPLACE INTO company_info VALUES (143160, 아이디스, 2020-10-29)
[2020-10-29 12:00] #1956 REPLACE INTO company_info VALUES (332370, 아이디피, 2020-10-29)
[2020-10-29 12:00] #1957 REPLACE INTO company_info VALUES (339950, 아이비김영, 2020-10-29)
[2020-10-29 12:00] #1958 REPLACE INTO company_info VALUES (052860, 아이앤씨, 2020-10-29)
[2020-10-29 12:00] #1959 REPLACE INTO company_info VALUES (069920, 아이에스이커머스, 2020-10-29)
[2020-10-29 12:00] #1960 REPLACE INTO company_info VALUES (101390, 아이엠, 2020-10-29)
[2020-10-29 12:00] #1961 REPLACE INTO company_info 

KeyError: 'pages_to_fetch'